<a href="https://colab.research.google.com/github/Gezahegn-Abd/Powe-allocation-far-near/blob/main/Far_near_NOMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def fair_power_allocation(args, pair_ue_m_index, pair_ue_dataset_m):

    #Far_j = args.Far_UEs_indx_all_models[l_m][uav_j] #np.linspace(0.5,10,10)  _all_models[l_m]
    #Near_j = args.Near_UEs_indx_all_models[l_m][uav_j]
    key=pair_ue_dataset_m.keys()

    aplha=0
    #this loop is considering maximum 2 multplexing on single subcarrier
    f_indx = pair_ue_dataset_m['F/N'][pair_ue_m_index].index[pair_ue_dataset_m['F/N'][pair_ue_m_index] == 'F'].tolist()
    n_indx = pair_ue_dataset_m['F/N'][pair_ue_m_index].index[pair_ue_dataset_m['F/N'][pair_ue_m_index] == 'N'].tolist()

    #PA(for comparison) Fixed
    #af = 0.75
    #an = 0.25

    alloc = False
    if len(f_indx)!=0 and len(n_indx)!=0:
        g_f = pair_ue_dataset_m[key[2]][f_indx[0]].tolist()
        R_f_max = pair_ue_dataset_m['R*'][f_indx[0]].tolist()  # %Far user target rate range (R*) r
        noise_f = pair_ue_dataset_m['N_0'][f_indx[0]].tolist()

        g_n = pair_ue_dataset_m[key[2]][n_indx[0]].tolist()

        epsilon = (2 ** (R_f_max/args.B)) - 1  #Target  SINR

        # compute delay based on the initially computed network dataset
        # count = args.beta[c].tolist().count(1) #count the number of subcarriers allocated for UAV_j 'F/N'


        I_inter=0
        #for far user   % BASIC   FAIR  PA %
        #aplha=epsilon * (noise_f + g_n* args.P_j_u+I_inter)/(args.P_j_u*(g_f + epsilon*g_n)) #-(I_inter*epsilon)
        aplha = np.divide(epsilon * ((g_f * args.P_j_u)+noise_f ), g_f * args.P_j_u * (1 + epsilon))


        if aplha>1:
            alpha_f=0  #far user will be in outage , entire power allocated for near UEs
        else:
            alpha_f = aplha
        alpha_n = 1 - alpha_f
        p_f=alpha_f * args.P_j_u
        p_n=alpha_n * args.P_j_u

        pair_ue_dataset_m.at[f_indx[0],'pi']=p_f
        pair_ue_dataset_m.at[f_indx[0], 'pigi'] = p_f * g_f
        pair_ue_dataset_m.at[n_indx[0], 'pi'] = p_n
        pair_ue_dataset_m.at[n_indx[0], 'pigi'] = p_n*g_n

        alloc=True

    elif len(f_indx)!=0 and len(n_indx)==0 and alloc == False:
        g_f = pair_ue_dataset_m[key[2]][f_indx[0]].tolist()
        R_f_max = pair_ue_dataset_m['R*'][f_indx[0]].tolist()  # %Far user target rate range (R*) r
        noise_f = pair_ue_dataset_m['N_0'][f_indx[0]].tolist()

        epsilon = (2 ** R_f_max/args.B) - 1 #Target  SINR

        #alpha_f = np.minimum(1, epsilon * (noise_f + g_f * args.P_j_u) / (g_f * args.P_j_u * (1 + epsilon) - (I_inter * epsilon)))
        #aplha = epsilon * (noise_f + I_inter) / (args.P_j_u * g_f)
        aplha = np.divide(epsilon * ((g_f * args.P_j_u)+noise_f ), g_f * args.P_j_u * (1 + epsilon))

        if aplha > 1:
            alpha_f = 0  # far user will be in outage , entire power allocated for near UEs
        else:
            alpha_f = aplha
        p_f = alpha_f * args.P_j_u
        pair_ue_dataset_m.at[f_indx[0], 'pi'] = p_f
        pair_ue_dataset_m.at[f_indx[0], 'pigi'] = p_f * g_f

        alloc = True
    elif len(f_indx)==0 and len(n_indx)!=0 and alloc == False:
        g_n = pair_ue_dataset_m[key[2]][n_indx[0]].tolist()

        #epsilon = np.round((2 ** R_n_max/args.B) - 1, 4) #Target  SINR

        #aaf = np.minimum(1, (epsilon * noise_n)/ (p2 * g2 * (1 + epsilon)))
        alpha_n= 1   #if there is no other UE paired the total transmit power is allocated for the single UE
        p_n=alpha_n * args.P_j_u
        pair_ue_dataset_m.at[n_indx[0], 'pi'] = p_n
        pair_ue_dataset_m.at[n_indx[0], 'pigi'] = p_n * g_n
        alloc = True
    return pair_ue_dataset_m